# Introduction to Hugging Face and Using its Models
Welcome to this session on using Hugging Face models! This is guide is designed to introduce you to the world of Hugging Face and empower you to leverage its powerful tools for various machine learning tasks.

## What is Hugging Face?
Hugging Face is an open-source platform that has revolutionized the field of Natural Language Processing (NLP) and is rapidly expanding into other domains like computer vision and audio. Its core mission is to democratize access to cutting-edge machine learning models and tools, making it easier for everyone to build and deploy AI applications.

Think of Hugging Face as a central hub for the ML community, offering:

*   **A vast Model Hub:** A repository of thousands of pre-trained models for various tasks, contributed by researchers and developers worldwide. You can find models for text classification, translation, summarization, image recognition, audio transcription, and much more.
*   **Powerful Libraries:** Open-source libraries like `transformers`, `datasets`, and `tokenizers` that provide easy-to-use interfaces for working with models, datasets, and text processing.
*   **A Collaborative Community:** A vibrant community of ML practitioners who share models, datasets, and expertise.

Hugging Face significantly reduces the barrier to entry for using state-of-the-art ML models, allowing you to quickly experiment and build applications without having to train models from scratch.

## Managing Hugging Face Tokens
To access some models or features on the Hugging Face Hub, you might need to use an API token. This token helps authenticate your requests and can be used to interact with the Hub programmatically, including downloading gated models or uploading your own.

Here's how you can manage and verify your Hugging Face token:
1.  **Obtain a Token:** Go to your Hugging Face profile settings (https://huggingface.co/settings/tokens) and generate a new access token. You can choose different roles for the token (e.g., read, write).
2.  **Store your Token Securely:** It's crucial to store your token securely. In Google Colab, you can use the "Secrets" feature (🔑 icon in the left panel) to store your token as an environment variable. Name your secret `HF_TOKEN`.
3.  **Log in Programmatically:** You can use the `huggingface_hub` library to log in to the Hugging Face Hub using your token.

Let's add a code block to install the necessary library and verify your token.

In [ ]:
%pip install huggingface_hub

In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import whoami

# Get your Hugging Face token from the .env file
load_dotenv()
hf_token = os.getenv("HF_TOKEN_AA")

# Authenticate with Hugging Face using the token
try:
    user_info = whoami(token=hf_token)
    print(f"Logged in as: {user_info['name']}")
except Exception as e:
    print(f"Error authenticating with Hugging Face: \n{e}")

## Showcasing Different Model Types

Hugging Face isn't just about text! Let's explore how to use models for other modalities like images and audio, and also how to work with datasets.

### Image Classification

#Image classification is the task of categorizing an image into one of several classes. We can use a pre-trained image classification model from the Hugging Face Hub.

In [ ]:
%pip install transformers pillow torch torchvision

In [ ]:
import torch
from transformers import pipeline
from PIL import Image
import requests
from io import BytesIO

# Determine device (use CPU for compatibility)
device = 0 if torch.cuda.is_available() else -1

# Load image classification pipeline with explicit device
classifier = pipeline("image-classification", device=device)

# Get an image from a URL using a public image (cat photo)
url = "https://images.unsplash.com/photo-1574158622682-e40e69881006?w=400"
headers = {"User-Agent": "Mozilla/5.0"}
try:
    response = requests.get(url, headers=headers, stream=True, timeout=10)
    response.raise_for_status()
    image = Image.open(BytesIO(response.content))

    # Classify the image
    predictions = classifier(image)

    print("Image classification results:")
    for prediction in predictions:
        print(f"- {prediction['label']}: {prediction['score']:.2f}")
except Exception as e:
    print(f"Error: {e}")
    # print("\nUsing a sample image instead...")

    # Create a simple test image if URL fails
    # test_image = Image.new('RGB', (224, 224), color='red')
    # predictions = classifier(test_image)
    # print("Image classification results:")
    # for prediction in predictions:
    # print(f"- {prediction['label']}: {prediction['score']:.2f}")

### Audio Classification

Audio classification is the task of categorizing audio data into different classes, such as identifying the type of sound or the speaker's emotion.

In [ ]:
%pip install soundfile

In [ ]:
from transformers import pipeline
import torch
import soundfile as sf

# Load an audio classification pipeline
classifier = pipeline("audio-classification", model="superb/wav2vec2-base-superb-ks")

# A simple sine wave, you would load your actual audio data
dummy_audio = torch.randn(16000)  # 1 second of random noise at 16kHz
sf.write("./content/dummy_audio.wav", dummy_audio.numpy(), 16000)

# Classify the audio
audio_file = "./content/dummy_audio.wav"
predictions = classifier(audio_file)

print("Audio classification results:")
for prediction in predictions:
    print(f"- {prediction['label']}: {prediction['score']:.2f}")

### Working with Datasets

Hugging Face provides the `datasets` library, which makes it easy to access and work with a wide variety of datasets for various ML tasks.

In [ ]:
%pip install datasets

In [ ]:
from datasets import load_dataset

# Load a dataset (e.g., the SQuAD dataset for question answering)
dataset = load_dataset("squad")

# Print information about the dataset
print(dataset)

# Access the first example in the training set
print("\nExample from the training set:")
print(dataset["train"][0])

## Transcription with Hugging Face

Audio transcription is the task of converting spoken language into text. Hugging Face also offers models for this task.

Here's how you can use a pre-trained model for audio transcription:

In [ ]:
from transformers import pipeline
import soundfile as sf
import torch

# Load the automatic speech recognition pipeline
transcriber = pipeline(
    "automatic-speech-recognition", model="facebook/wav2vec2-base-960h"
)

# Transcribe the audio file
audio_file = "./content/harvard.wav"
try:
    transcription = transcriber(audio_file)
    print("Transcription result:")
    print(transcription["text"])
except Exception as e:
    print(f"Error transcribing audio: \n{e}")

## Summarization with Hugging Face

Text summarization is the task of creating a shorter version of a text while preserving its main ideas. Hugging Face provides several models that can be used for this purpose.

Here's how you can use a pre-trained model from Hugging Face for summarization:

In [ ]:
from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("text-classification")

# Text to summarize
text = """
Based on coalescence of Mitochondrial DNA and Y Chromosome data, it is thought that the earliest extant lineages of anatomically modern humans or Homo sapiens on the Indian subcontinent had reached there from Africa between 80,000 and 50,000 years ago
"""

# Summarize the text
summary = summarizer(text, max_length=100, min_length=30, do_sample=False)

print("\nOriginal Text:")
print(text)
print("\nSummary:")
print(summary[0]["label"])